In [73]:
import numpy as np
import matplotlib.pyplot as plt
import copy
import scipy.linalg as la

In [74]:
#parameter definitions
k_OH = 450
k_HOH = 55

r0_OH = 0.9572
theta0_HOH = 104.52

m_O = 15.9994
m_H = 1.008
c_O = -0.834
c_H = 0.417
mass_vec = np.array([m_O, m_O, m_O, m_H, m_H, m_H, m_H, m_H, m_H])

In [75]:
x_O = -23.107 
y_O = 18.401
z_O = -21.626
x_H1 = -22.157 
y_H1 = 18.401
z_H1 = -21.52
x_H2 = -23.424 
y_H2 = 18.401
z_H2 = -20.730


In [76]:
#get theta
def getAngle(vec1, vec2):
    c = np.dot(vec1,vec2)/np.linalg.norm(vec1)/np.linalg.norm(vec2) # -> cosine of the angle
    angle = np.arccos(c) # if you really want the angle
    return (180/np.pi)*angle

In [77]:
def getU(x_O, y_O, z_O, x_H1, y_H1, z_H1, x_H2, y_H2, z_H2):
    r_O = np.array([x_O, y_O, z_O])
    r_H1 = np.array([x_H1, y_H1, z_H1])
    r_H2 = np.array([x_H2, y_H2, z_H2])
#     print("angle", getAngle(r_O-r_H1, r_O-r_H2)-theta0_HOH )
    U = (1/2)*k_OH*(np.linalg.norm(r_H1-r_O)-r0_OH)**2 + \
        (1/2)*k_OH*(np.linalg.norm(r_H2-r_O)-r0_OH)**2 + \
        (1/2)*k_HOH*(getAngle(r_O-r_H1, r_O-r_H2)-theta0_HOH)**2
    return U
    

In [78]:
def getGradient(x, index, f):
    eps = 0.00000000001 
    xx0 = 1. * x[index]
    f0 = f(*x)
    x[index] = x[index] + eps
    f1 = f(*x)
#     print("eps", eps, "f0", f0, "f1", f1)
    gradient = (f1 - f0)/eps
    x[index] = xx0
    return gradient



In [79]:
pos = [x_O, y_O, z_O, x_H1, y_H1, z_H1, x_H2, y_H2, z_H2]
getU(x_O, y_O, z_O, x_H1, y_H1, z_H1, x_H2, y_H2, z_H2)

54.154460529609636

In [80]:
def gradient_decent(x, indexs, steps, epsilon, f):
    for j in range(steps):
        grad = np.zeros(9)
        for i in indexs:
            grad[i] = (getGradient(x, i, f))
#             print("grad:", grad, "x", x)
        for i in indexs:
            x[i] = x[i] - grad[i]*epsilon
#             grad = getGradient(x, index, f)
#         print("grad:", grad, "x", x)
    return x

In [81]:
pos = [x_O, y_O, z_O, x_H1, y_H1, z_H1, x_H2, y_H2, z_H2]
pos = gradient_decent(pos, [3,5,6,8], 70000, 0.000001, getU)


In [82]:
U = getU(*pos)
print("equilibrium_pos", pos, "U:", U, "angle:", getAngle(np.array([pos[0],pos[1],pos[2]])-np.array([pos[3],pos[4],pos[5]]),np.array([pos[0],pos[1],pos[2]])-np.array([pos[6],pos[7],pos[8]]) ))

equilibrium_pos [-23.107, 18.401, -21.626, -22.154481695535658, 18.401, -21.531444623628232, -23.43734871848534, 18.401, -20.727611684739887] U: 5.445839071432381e-17 angle: 104.5200000002777


In [83]:
    #Hessian Matrix
    def getHessian (pos, getU):
        hessian = np.zeros((9, 9))
        for i in range(9):
            for j in range(9):
                x = copy.deepcopy(pos)
                gd_0 = getGradient( x, j, getU)
                eps = 0.00000000001
                x[i] = x[i]+eps
                gd_1 = getGradient( x, j, getU)

                double_g = (gd_1 - gd_0)/eps
                hessian[i][j] = double_g
        return hessian

In [84]:
hessian = getHessian(pos, getU)
for i in range(9):
    for j in range(9):
        hessian[i][j] = hessian[i][j]/np.sqrt(mass_vec[i]*mass_vec[j])
print("hessian:",hessian)

hessian: [[  8718.66458613      0.          13858.44880105   3960.15153752
       0.         -41024.62869862 -38696.06367126      0.
  -14187.70800161]
 [     0.              0.              0.              0.
       0.              0.              0.              0.
       0.        ]
 [ 13858.44880105      0.          22152.67656284   6486.336951
       0.         -65457.09737994 -61700.26604926      0.
  -22801.34360223]
 [  3960.15153752      0.           6486.336951     2349.86023805
       0.         -19176.00819478 -18127.75340585      0.
   -6665.68730592]
 [     0.              0.              0.              0.
       0.              0.              0.              0.
       0.        ]
 [-41024.62869862      0.         -65457.09737994 -19176.00819478
       0.         193630.67427453 182623.71906268      0.
   67151.8736306 ]
 [-38696.06367126      0.         -61700.26604926 -18127.75340585
       0.         182623.71906268 172299.51293358      0.
   63191.415726  ]
 [     0

In [85]:
eigvals, eigvac = la.eig(hessian)

In [86]:
print("eigen_values:", eigvals.real)

eigen_values: [ 4.21886262e+05  4.82013599e+02  4.66445159e+02 -1.03147669e+00
  3.09998268e-01 -1.60686015e-02  0.00000000e+00  0.00000000e+00
  0.00000000e+00]


In [87]:
print("eigen_vectors:",eigvac)

eigen_vectors: [[-0.14350514  0.22568722  0.11167725  0.15807513  0.62119443  0.71072461
   0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   1.          0.          0.        ]
 [-0.22902999 -0.15027352  0.16448213 -0.86439578 -0.19320521  0.33674983
   0.          0.          0.        ]
 [-0.06720282 -0.65656575 -0.71240728  0.00394467  0.19795129  0.13296915
   0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          1.          0.        ]
 [ 0.67746431 -0.06690619 -0.03093223  0.14262612 -0.47033786  0.54226281
   0.          0.          0.        ]
 [ 0.63894878 -0.24299122  0.2685813  -0.29866252  0.55401705 -0.2538299
   0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          1.        ]
 [ 0.23499339  0.65717836 -0.61632537 -0.34391151  0.09722099 -0.07287457
   0.          0.          0.   

In [88]:
freq = np.sqrt(eigvals)
print(freq.real)

[6.49527722e+02 2.19548081e+01 2.15973415e+01 0.00000000e+00
 5.56774881e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00]


So the frequencies are 6.49527722e+02 2.19548081e+01 2.15973415e+01

This code optimises the coordinates of the water molecule using gradient decent. This I have computed the hessian matrix and diagonalized it to get eigenvalues. (I have not accounted for masses for now so this is not same as actual values)
Note: also the gradient decent is not powerful enough to minimise U by a lot. So there is already a significant deviation from the equilibrium value. So probably that is the reason I am not getting correct eigenvalues.